# Heuristic Candidate Mining - 페이지 필터링

메트릭별로 관련 키워드가 있는 페이지만 추출하여 주석 작업 범위를 줄입니다.

## 목적
- 각 메트릭에 관련된 페이지 번호만 추출
- 관련 없는 페이지는 제외하여 작업 효율 향상

## 1. 라이브러리 임포트

In [29]:
import json
import os
from pathlib import Path

# 프로젝트 경로 설정
PROJECT_DIR = Path('../myproj')
METRIC_FILE = PROJECT_DIR / 'metric_sid_map.json'
PAGES_FILE = PROJECT_DIR / 'pages' / 'metadata.json'
OUTPUT_FILE = PROJECT_DIR / 'metric_page_mapping.json'

print(f"프로젝트 디렉토리: {PROJECT_DIR}")
print(f"메트릭 파일: {METRIC_FILE}")
print(f"페이지 파일: {PAGES_FILE}")
print(f"결과 저장 경로: {OUTPUT_FILE}")

프로젝트 디렉토리: ../myproj
메트릭 파일: ../myproj/metric_sid_map.json
페이지 파일: ../myproj/pages/metadata.json
결과 저장 경로: ../myproj/metric_page_mapping.json


## 2. 데이터 로드

In [30]:
# 메트릭 데이터 로드
with open(METRIC_FILE, 'r', encoding='utf-8') as f:
    metrics = json.load(f)

print(f"로드된 메트릭 개수: {len(metrics)}")
print("\n메트릭 목록:")
for metric_id, metric_info in metrics.items():
    print(f"  - {metric_id}: {metric_info['topic']}")

로드된 메트릭 개수: 9

메트릭 목록:
  - TC-HW-230a.1: Product Security
  - TC-HW-330a.1: Employee Diversity & Inclusion
  - TC-HW-410a.1: Product Lifecycle Management
  - TC-HW-410a.2: Product Lifecycle Management
  - TC-HW-410a.3: Product Lifecycle Management
  - TC-HW-410a.4: Product Lifecycle Management
  - TC-HW-430a.1: Supply Chain Management
  - TC-HW-430a.2: Supply Chain Management
  - TC-HW-440a.1: Materials Sourcing


In [31]:
# 페이지 메타데이터 로드
with open(PAGES_FILE, 'r', encoding='utf-8') as f:
    pages = json.load(f)

print(f"로드된 페이지 개수: {len(pages)}")
print(f"페이지 범위: {pages[0]['page']} ~ {pages[-1]['page']}")

로드된 페이지 개수: 83
페이지 범위: 1 ~ 83


## 3. 메트릭별 키워드 정의

각 메트릭과 관련된 키워드를 정의합니다.

In [ ]:
# 메트릭별 키워드 매핑 (SASB Hardware 메트릭)
# metric_keywords.py에서 임포트 (강제 리로드)
import sys
sys.path.insert(0, '../myproj')

# 이미 임포트된 경우 리로드
import importlib
if 'metric_keywords' in sys.modules:
    import metric_keywords
    importlib.reload(metric_keywords)
    from metric_keywords import METRIC_KEYWORDS
else:
    from metric_keywords import METRIC_KEYWORDS

print("메트릭별 키워드 매핑:")
for metric_id, keywords in METRIC_KEYWORDS.items():
    print(f"\n{metric_id}: {len(keywords)}개 키워드")
    print(f"  첫 8개: {', '.join(keywords[:8])}...")
    print(f"  총 키워드: 영어 + 한국어")

## 4. 페이지 필터링 함수

In [24]:
def find_related_pages(metric_id, keywords, pages):
    """
    특정 메트릭에 대해 관련 키워드가 있는 페이지 번호를 반환합니다.
    
    Parameters:
    -----------
    metric_id : str
        메트릭 ID
    keywords : List[str]
        검색할 키워드 리스트
    pages : List[Dict]
        페이지 메타데이터 리스트
    
    Returns:
    --------
    List[int]
        관련된 페이지 번호 리스트 (정렬됨)
    """
    related_pages = set()
    
    for page_meta in pages:
        text = page_meta.get('text', '').lower()
        page_num = page_meta.get('page')
        
        # 키워드가 하나라도 있으면 해당 페이지 추가
        for keyword in keywords:
            if keyword.lower() in text:
                related_pages.add(page_num)
                break  # 하나라도 매칭되면 다음 페이지로
    
    return sorted(list(related_pages))

print("페이지 필터링 함수 정의 완료")

페이지 필터링 함수 정의 완료


## 5. 메트릭별 관련 페이지 추출

In [25]:
# 각 메트릭별로 관련 페이지 찾기
metric_page_mapping = {}

print("="*60)
print("메트릭별 관련 페이지 추출")
print("="*60)

for metric_id, keywords in METRIC_KEYWORDS.items():
    related_pages = find_related_pages(metric_id, keywords, pages)
    metric_page_mapping[metric_id] = related_pages
    
    metric_info = metrics[metric_id]
    print(f"\n{metric_id}: {metric_info['topic']}")
    print(f"  관련 페이지: {related_pages}")
    print(f"  총 {len(related_pages)}개 페이지")

print("\n" + "="*60)

메트릭별 관련 페이지 추출

TC-HW-230a.1: Product Security
  관련 페이지: [10, 48, 49, 53, 54, 57]
  총 6개 페이지

TC-HW-330a.1: Employee Diversity & Inclusion
  관련 페이지: [10, 31, 32, 34, 36, 37, 48, 49, 53]
  총 9개 페이지

TC-HW-410a.1: Product Lifecycle Management
  관련 페이지: [10, 17, 21, 27, 28, 49, 53, 54, 57, 79, 80]
  총 11개 페이지

TC-HW-410a.2: Product Lifecycle Management
  관련 페이지: [79]
  총 1개 페이지

TC-HW-410a.3: Product Lifecycle Management
  관련 페이지: [79]
  총 1개 페이지

TC-HW-410a.4: Product Lifecycle Management
  관련 페이지: [15]
  총 1개 페이지

TC-HW-430a.1: Supply Chain Management
  관련 페이지: [6, 9, 17, 31, 34, 35, 36, 37, 39, 42, 50, 57, 79]
  총 13개 페이지

TC-HW-430a.2: Supply Chain Management
  관련 페이지: []
  총 0개 페이지

TC-HW-440a.1: Materials Sourcing
  관련 페이지: [8, 10, 32, 38, 48, 79, 80, 82]
  총 8개 페이지



## 6. 결과 요약

In [26]:
print("\n📊 결과 요약")
print("="*60)

# 전체 통계
total_pages = len(pages)
all_related_pages = set()
for pages_list in metric_page_mapping.values():
    all_related_pages.update(pages_list)

print(f"\n전체 페이지 수: {total_pages}")
print(f"관련 페이지 수: {len(all_related_pages)}")
print(f"제외된 페이지 수: {total_pages - len(all_related_pages)}")
print(f"작업 효율: {len(all_related_pages)/total_pages*100:.1f}% 페이지만 주석 작업 필요")

# 메트릭별 상세
print("\n\n메트릭별 상세:")
print("-" * 60)
for metric_id, pages_list in metric_page_mapping.items():
    metric_info = metrics[metric_id]
    print(f"\n{metric_id}")
    print(f"  주제: {metric_info['topic']}")
    print(f"  카테고리: {metric_info['category']}")
    print(f"  관련 페이지: {pages_list}")
    print(f"  페이지 수: {len(pages_list)}")

# 제외된 페이지
excluded_pages = sorted(set(range(1, total_pages + 1)) - all_related_pages)
if excluded_pages:
    print(f"\n\n❌ 제외된 페이지 (모든 메트릭에서 키워드 없음):")
    print(f"  {excluded_pages}")
else:
    print(f"\n\n✅ 모든 페이지에 최소 하나의 메트릭 키워드가 있습니다.")


📊 결과 요약

전체 페이지 수: 83
관련 페이지 수: 27
제외된 페이지 수: 56
작업 효율: 32.5% 페이지만 주석 작업 필요


메트릭별 상세:
------------------------------------------------------------

TC-HW-230a.1
  주제: Product Security
  카테고리: Discussion and Analysis
  관련 페이지: [10, 48, 49, 53, 54, 57]
  페이지 수: 6

TC-HW-330a.1
  주제: Employee Diversity & Inclusion
  카테고리: Quantitative
  관련 페이지: [10, 31, 32, 34, 36, 37, 48, 49, 53]
  페이지 수: 9

TC-HW-410a.1
  주제: Product Lifecycle Management
  카테고리: Quantitative
  관련 페이지: [10, 17, 21, 27, 28, 49, 53, 54, 57, 79, 80]
  페이지 수: 11

TC-HW-410a.2
  주제: Product Lifecycle Management
  카테고리: Quantitative
  관련 페이지: [79]
  페이지 수: 1

TC-HW-410a.3
  주제: Product Lifecycle Management
  카테고리: Quantitative
  관련 페이지: [79]
  페이지 수: 1

TC-HW-410a.4
  주제: Product Lifecycle Management
  카테고리: Quantitative
  관련 페이지: [15]
  페이지 수: 1

TC-HW-430a.1
  주제: Supply Chain Management
  카테고리: Quantitative
  관련 페이지: [6, 9, 17, 31, 34, 35, 36, 37, 39, 42, 50, 57, 79]
  페이지 수: 13

TC-HW-430a.2
  주제: Supply Chain Management

## 7. JSON 파일로 저장

In [27]:
# 결과를 JSON 파일로 저장
output_data = {
    "metric_page_mapping": metric_page_mapping,
    "summary": {
        "total_pages": total_pages,
        "related_pages": len(all_related_pages),
        "excluded_pages": total_pages - len(all_related_pages),
        "efficiency": f"{len(all_related_pages)/total_pages*100:.1f}%"
    },
    "excluded_page_list": excluded_pages
}

with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✅ 결과 저장 완료: {OUTPUT_FILE}")
print(f"\n파일 내용:")
print(json.dumps(output_data, indent=2, ensure_ascii=False))

✅ 결과 저장 완료: ../myproj/metric_page_mapping.json

파일 내용:
{
  "metric_page_mapping": {
    "TC-HW-230a.1": [
      10,
      48,
      49,
      53,
      54,
      57
    ],
    "TC-HW-330a.1": [
      10,
      31,
      32,
      34,
      36,
      37,
      48,
      49,
      53
    ],
    "TC-HW-410a.1": [
      10,
      17,
      21,
      27,
      28,
      49,
      53,
      54,
      57,
      79,
      80
    ],
    "TC-HW-410a.2": [
      79
    ],
    "TC-HW-410a.3": [
      79
    ],
    "TC-HW-410a.4": [
      15
    ],
    "TC-HW-430a.1": [
      6,
      9,
      17,
      31,
      34,
      35,
      36,
      37,
      39,
      42,
      50,
      57,
      79
    ],
    "TC-HW-430a.2": [],
    "TC-HW-440a.1": [
      8,
      10,
      32,
      38,
      48,
      79,
      80,
      82
    ]
  },
  "summary": {
    "total_pages": 83,
    "related_pages": 27,
    "excluded_pages": 56,
    "efficiency": "32.5%"
  },
  "excluded_page_list": [
    1,
    2,
    3,


## 8. UI에서 사용할 수 있는 형태로 출력

In [28]:
print("\n" + "="*60)
print("UI 통합 가이드")
print("="*60)

print("\n1️⃣  생성된 파일 위치:")
print(f"   {OUTPUT_FILE}")

print("\n2️⃣  UI에서 사용하는 방법:")
print("""\n   # ui.py에서 로드
   with open('myproj/metric_page_mapping.json', 'r') as f:
       mapping = json.load(f)
   
   metric_pages = mapping['metric_page_mapping']
   
   # 특정 메트릭의 관련 페이지만 표시
   metric_id = "metric1"
   pages_to_annotate = metric_pages[metric_id]
   print(f"{metric_id}의 주석 대상 페이지: {pages_to_annotate}")
""")

print("\n3️⃣  다음 작업:")
print("   - metric_page_mapping.json을 UI 코드에 통합")
print("   - 메트릭 선택 시 관련 페이지만 네비게이션에 표시")
print("   - 관련 없는 페이지는 건너뛰기")

print("\n" + "="*60)


UI 통합 가이드

1️⃣  생성된 파일 위치:
   ../myproj/metric_page_mapping.json

2️⃣  UI에서 사용하는 방법:

   # ui.py에서 로드
   with open('myproj/metric_page_mapping.json', 'r') as f:
       mapping = json.load(f)
   
   metric_pages = mapping['metric_page_mapping']
   
   # 특정 메트릭의 관련 페이지만 표시
   metric_id = "metric1"
   pages_to_annotate = metric_pages[metric_id]
   print(f"{metric_id}의 주석 대상 페이지: {pages_to_annotate}")


3️⃣  다음 작업:
   - metric_page_mapping.json을 UI 코드에 통합
   - 메트릭 선택 시 관련 페이지만 네비게이션에 표시
   - 관련 없는 페이지는 건너뛰기

